# DATAFRAME
### (DataFrame)

# 1. DataFrame

### Load DataFrame formatter (probably will be included in the next release of Microsoft.Data.Analysis)

In [1]:
#r "nuget:Microsoft.Data.Analysis"
using Microsoft.AspNetCore.Html;
using Microsoft.Data.Analysis;

Installed package Microsoft.Data.Analysis version 0.4.0

In [1]:
#load "C:\Users\dcost\source\repos\SmartFireAlarm\SmartFireAlarm\Jupyter\Formatters.csx"
Formatters.Categories = new string[] { "FlashLight", "Infrared", "DayLight", "Lighter" };
Formatters.Register<DataFrame>();

Error: C:\Users\dcost\source\repos\SmartFireAlarm\SmartFireAlarm\Jupyter\Formatters.csx(23,52): error CS0246: The type or namespace name 'ConfusionMatrix' could not be found (are you missing a using directive or an assembly reference?)
C:\Users\dcost\source\repos\SmartFireAlarm\SmartFireAlarm\Jupyter\Formatters.csx(27,54): error CS0246: The type or namespace name 'MulticlassClassificationMetrics' could not be found (are you missing a using directive or an assembly reference?)
C:\Users\dcost\source\repos\SmartFireAlarm\SmartFireAlarm\Jupyter\Formatters.csx(31,61): error CS0246: The type or namespace name 'TrainCatalogBase' could not be found (are you missing a using directive or an assembly reference?)
C:\Users\dcost\source\repos\SmartFireAlarm\SmartFireAlarm\Jupyter\Formatters.csx(31,100): error CS0246: The type or namespace name 'MulticlassClassificationMetrics' could not be found (are you missing a using directive or an assembly reference?)
C:\Users\dcost\source\repos\SmartFireAl

In [1]:

using Microsoft.Data.Analysis;
using Microsoft.AspNetCore.Html;
using ApexCode.Interactive.Formatting;


In [1]:

Formatters.Register<DataFrame>();

DataFrame formatter loaded.


In [1]:
using System.Globalization;
using System.IO;

private const int DefaultStreamReaderBufferSize = 1024;

public static void WriteCsv(DataFrame dataFrame, Stream csvStream,
                           char separator = ',', bool header = true,
                           Encoding encoding = null, CultureInfo cultureInfo = null)
        {
            if (cultureInfo is null)
            {
                cultureInfo = CultureInfo.CurrentCulture;
            }

            if (cultureInfo.NumberFormat.NumberDecimalSeparator.Equals(separator.ToString()))
            {
                throw new ArgumentException("Decimal separator cannot match the column separator");
            }

            if (encoding is null)
            {
                encoding = Encoding.ASCII;
            }

            using (StreamWriter csvFile = new StreamWriter(csvStream, encoding, bufferSize: DefaultStreamReaderBufferSize, leaveOpen: true))
            {
                if (dataFrame != null)
                {
                    var columnNames = new List<string>();
                    
                    foreach (var col in dataFrame.Columns)
                    {
                        columnNames.Add(col.Name);
                    }

                    if (header)
                    {
                        var headerColumns = string.Join(separator.ToString(), columnNames);
                        csvFile.WriteLine(headerColumns);
                    }

                    var record = new StringBuilder();

                    foreach (var row in dataFrame.Rows)
                    {
                        bool firstRow = true;
                        foreach (var cell in row)
                        {
                            if (!firstRow)
                            {
                                record.Append(separator);
                            }
                            else
                            {
                                firstRow = false;
                            }

                            Type t = cell?.GetType();

                            if (t == typeof(bool))
                            {
                                record.AppendFormat(cultureInfo, "{0}", cell);
                                continue;
                            }

                            if (t == typeof(float))
                            {
                                record.AppendFormat(cultureInfo, "{0:G9}", cell);
                                continue;
                            }

                            if (t == typeof(double))
                            {
                                record.AppendFormat(cultureInfo, "{0:G17}", cell);
                                continue;
                            }

                            if (t == typeof(decimal))
                            {
                                record.AppendFormat(cultureInfo, "{0:G31}", cell);
                                continue;
                            }

                            record.Append(cell);
                        }

                        csvFile.WriteLine(record);

                        record.Clear();
                    }
                }
            }
        }

In [1]:
public static void WriteCsv(DataFrame dataFrame, string path,
                                   char separator = ',', bool header = true,
                                   Encoding encoding = null, CultureInfo cultureInfo = null)
        {
            using (System.IO.FileStream csvStream = new FileStream(path, FileMode.Create))
            {
                WriteCsv(dataFrame: dataFrame, csvStream: csvStream,
                           separator: separator, header: header,
                           encoding: encoding, cultureInfo: cultureInfo);
            }
        }

### Load data into data frame

In [1]:
const string SENSORS_DATASET_PATH = "./sensors.csv";
const string LABELS_DATASET_PATH = "./labels.csv";
var df1 = DataFrame.LoadCsv(SENSORS_DATASET_PATH);
var df2 = DataFrame.LoadCsv(LABELS_DATASET_PATH);

### prepare sensors new df


In [1]:
var sensorsCreatedAt = df1.Columns["CreatedAt"] as IEnumerable<string>;
var sensorsTicksList = sensorsCreatedAt.ToList().Select(d => DateTime.Parse(d.ToString()).Ticks / 10000000);

In [1]:
var sensorsTicks = new PrimitiveDataFrameColumn<long>("Id", sensorsTicksList);
var df11 = new DataFrame(df1.Columns["Temperature"], df1.Columns["Luminosity"], df1.Columns["Infrared"], df1.Columns["Distance"], df1.Columns["CreatedAt"], sensorsTicks);

### prepare labels new df

In [1]:
var labelsCreatedAt = df2.Columns["CreatedAt"] as IEnumerable<string>;
var labelsTicksList = labelsCreatedAt.ToList().Select(d => DateTime.Parse(d.ToString()).Ticks / 10000000);

In [1]:
using System.Linq;

var labelsTicks = new PrimitiveDataFrameColumn<long>("Id", labelsTicksList);
var df22 = new DataFrame(labelsTicks, df2.Columns["CreatedAt"], df2.Columns["Label"], df2.Columns["Score"]);

### merging df1 and df2 into df

In [1]:
var df = df11.Merge<long>(df22, "Id", "Id", "_sensor", "_label", JoinAlgorithm.FullOuter);

In [1]:
var cleaned = df.DropNulls();
WriteCsv(cleaned, "merged.csv");

In [1]:
using XPlot.Plotly;

display(Chart.Plot(
    new Graph.Histogram()
    {
        x = df22.Columns["Label"]
        
    }
));